# Snowflake Online Feature Store:  Predicting Taxi Trip Durations

This notebook demonstrates how to use the Snowflake Online Feature Store and Snowpark ML to build features, register feature views, and train a model to predict taxi trip durations in New York City. It covers entity and feature view registration, feature engineering, online and offline store usage, and real-time inference.


In [ ]:
import pandas as pd
from snowflake.ml.feature_store import CreationMode, FeatureStore, feature_view
from snowflake.ml.feature_store.entity import Entity
from snowflake.snowpark import Session
from snowflake.snowpark.exceptions import SnowparkSQLException
from snowflake.snowpark.functions import avg, col, datediff, dayofweek, hour, month, nullifzero, when
from snowflake.snowpark.types import DecimalType
pd.set_option('display.max_columns', None)

In [ ]:
# Snowflake connection parameters
connection_parameters = {
    "account": "<account_name>",
    "user": "<username>",
    "password": "<programmatic_access_token>",
    "role": "<role>",
    "host": "<host - if using private link>",
    "warehouse": "<warehouse>",
    "database": "<database>",
    "schema": "<schema>"
}
session = Session.builder.configs(connection_parameters).create()

TAXI_DB = session.get_current_database()
TAXI_SCHEMA = session.get_current_schema()
TAXI_TABLE = "NYC_YELLOW_TRIPS"
TAXI_TABLE_FULL_NAME = f"{TAXI_DB}.{TAXI_SCHEMA}.{TAXI_TABLE}"

## Data Setup
Load a sample dataset of NYC taxi trips. The data includes pickup and dropoff location IDs, timestamps, fare amounts, and more.

In [ ]:
# One-time setup: load example data
try:
    session.table(TAXI_TABLE_FULL_NAME).limit(0).collect()
    print("NYC taxi table already exists")
except SnowparkSQLException as e:
    print("Loading NYC taxi table")
    from snowflake.ml.feature_store.examples.example_helper import ExampleHelper
    example_helper = ExampleHelper(session, session.get_current_database(), session.get_current_schema())
    source_tables = example_helper.load_example('new_york_taxi_features')
    for table in source_tables:
        print(f"{table}:")
        df = session.table(table).limit(5).to_pandas()
        print(df)

## Feature Engineering with Snowflake Feature Store

In [ ]:
# Set up Feature Store and context
fs = FeatureStore(
    session=session,
    database=TAXI_DB,
    name=TAXI_SCHEMA,
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

## Register Entities
Entities represent the keys used to join features. Here we define entities for location, trip, and route.


In [ ]:
# Define and register entities
route_entity = Entity(
    name="route",
    join_keys=["PULOCATIONID", "DOLOCATIONID"],
    desc="A taxi route defined by pickup and dropoff location IDs."
)
pickup_time_entity = Entity(
    name="pickup_time",
    join_keys=["PICKUP_HOUR", "PICKUP_DAY_OF_WEEK"],
    desc="Pickup time bucketed by hour and day of week"
)

fs.register_entity(route_entity)
fs.register_entity(pickup_time_entity)
print("List Entities:")
fs.list_entities().show()

## Load Data and Create Features
Read the NYC taxi trip data and engineer features for ETA, speed, rush hour, and more.


In [ ]:
# Load data
df = session.table(TAXI_TABLE_FULL_NAME)

# Create features
df = df.with_column("TRIP_DISTANCE_INT", col("TRIP_DISTANCE").cast(DecimalType(10, 0)))
df = df.with_column("ETA_MINUTES", (datediff("second", col("TPEP_PICKUP_DATETIME"), col("TPEP_DROPOFF_DATETIME")) / 60.0))
df = df.with_column("ETA_MINUTES_INT", col("ETA_MINUTES").cast(DecimalType(10, 0)))
df = df.with_column("PICKUP_HOUR", hour(col("TPEP_PICKUP_DATETIME")))
df = df.with_column("PICKUP_DAY_OF_WEEK", dayofweek(col("TPEP_PICKUP_DATETIME")))  # 0=Sunday, 6=Saturday
df = df.with_column("IS_WEEKEND", when(col("PICKUP_DAY_OF_WEEK").isin([0, 6]), 1).otherwise(0))
df = df.with_column("PICKUP_MONTH", month(col("TPEP_PICKUP_DATETIME")))
df = df.with_column("SPEED_MPH", df["TRIP_DISTANCE"] / nullifzero((df["ETA_MINUTES"] / 60)))
df = df.with_column("SPEED_MPH_INT", col("SPEED_MPH").cast(DecimalType(10, 0)))
df = df.with_column("IS_RUSH_HOUR", when((col("PICKUP_HOUR").isin([7, 8, 9, 16, 17, 18, 19])),1).otherwise(0))
df = df.with_column("IS_LOOP_ROUTE", when(col("PULOCATIONID") == col("DOLOCATIONID"), 1).otherwise(0))
print("Trip Data with Basic Features:")
df.show(n=10)

In [ ]:
# Aggregate features by route and pickup time bucket (hour x day-of-week)
agg_route_and_time = (
    df.group_by("PULOCATIONID", "DOLOCATIONID", "PICKUP_HOUR", "PICKUP_DAY_OF_WEEK")
      .agg(
          avg("ETA_MINUTES_INT").alias("AVG_ETA_ROUTE"),
          avg("SPEED_MPH_INT").alias("AVG_SPEED_ROUTE"),
      )
)
df = df.join(
    agg_route_and_time,
    on=["PULOCATIONID", "DOLOCATIONID", "PICKUP_HOUR", "PICKUP_DAY_OF_WEEK"],
    how="left"
)
agg_route = (
    df.group_by("PULOCATIONID", "DOLOCATIONID")
      .agg(
          avg("TRIP_DISTANCE_INT").alias("AVG_DISTANCE_ROUTE"),
      )
)
df = df.join(agg_route, on=["PULOCATIONID", "DOLOCATIONID"], how="left")

print("Trip Data with Aggregated Features:")
df.show()

In [ ]:
# Select features for the feature view
feature_df = df.select(
    # Columns direct from table
    "PULOCATIONID", "DOLOCATIONID", "TRIP_DISTANCE",
    "VENDORID", "FARE_AMOUNT", "TOTAL_AMOUNT", "TPEP_PICKUP_DATETIME",
    "TIP_AMOUNT", "TOLLS_AMOUNT",
    # Feature columns created from table (exclude final label ETA_MINUTES)
    "PICKUP_HOUR", "PICKUP_DAY_OF_WEEK", "IS_WEEKEND", "PICKUP_MONTH",
    "SPEED_MPH", "IS_RUSH_HOUR", "IS_LOOP_ROUTE",
    # Feature columns created from aggregating by column combinations
    "AVG_ETA_ROUTE", "AVG_DISTANCE_ROUTE", "AVG_SPEED_ROUTE"
)

## Register Feature View
Create and register a feature view for trip-based features, enabling online serving.


In [ ]:
# Define and register the feature view
route_fv = feature_view.FeatureView(
    name="nyc_taxi_trip_fv",
    entities=[route_entity, pickup_time_entity],
    feature_df=feature_df,
    timestamp_col="TPEP_PICKUP_DATETIME",
    refresh_freq="60s", # Dynamic Table refresh minimum
    desc="Trip-based features for taxi ETA prediction",
    online_config=feature_view.OnlineConfig(enable=True, target_lag="10s"),
)

registered_route_fv = fs.register_feature_view(route_fv, "v1", overwrite=True)

print("Registered feature view:", registered_route_fv.name, registered_route_fv.version)
print("Online feature table:", registered_route_fv.fully_qualified_online_table_name())

In [ ]:
# Check refresh history
fs.get_refresh_history(registered_route_fv, store_type=feature_view.StoreType.ONLINE).show()

In [ ]:
# Fetch features from the online store
online_df = fs.read_feature_view(
    registered_route_fv,
    store_type=feature_view.StoreType.ONLINE,
)
online_df.show()

In [ ]:
# Explore online tables
fs.list_feature_views().show()

## Train/Test Split
Split the data into training and test sets for ML model development.


In [ ]:
# Create train/test split for ML
spine_cols = [
    "ETA_MINUTES",           # label
    "PULOCATIONID",          # join key
    "DOLOCATIONID",          # join key
    "PICKUP_HOUR",           # join key
    "PICKUP_DAY_OF_WEEK",    # join key
    "TPEP_PICKUP_DATETIME"   # for point-in-time correctness (as spine_timestamp_col)
]
train_spine_df, test_spine_df = df.select(spine_cols).random_split([0.85, 0.15], seed=42)

In [ ]:
# Generate training set with features for train and test
train_set = fs.generate_training_set(
    spine_df=train_spine_df,
    features=[registered_route_fv],
    spine_label_cols=["ETA_MINUTES"],  # Target column for regression
    save_as="TAXI_TRAIN_SET",
    spine_timestamp_col="TPEP_PICKUP_DATETIME"
)
test_set = fs.generate_training_set(
    spine_df=test_spine_df,
    features=[registered_route_fv],
    spine_label_cols=["ETA_MINUTES"],
    save_as="TAXI_TEST_SET",
    spine_timestamp_col="TPEP_PICKUP_DATETIME"
)

In [ ]:
print("Train set sample:")
print(train_set.limit(5).to_pandas())
print("Test set sample:")
print(test_set.limit(5).to_pandas())
print(f"Train set count: {train_set.count()}")
print(f"Test set count: {test_set.count()}")

## Train XGBoost Model
Train an XGBoost regressor using Snowpark ML on the generated training set.


In [ ]:
# Train XGBoost model using Snowpark ML
from snowflake.ml.modeling.xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define feature columns (exclude label and keys)
feature_columns = [
    col for col in train_set.columns
    if col not in ["ETA_MINUTES", "TPEP_PICKUP_DATETIME"]
]
label_column = "ETA_MINUTES"

regressor = XGBRegressor(
    input_cols=feature_columns,
    label_cols=[label_column],
    output_cols=["predicted_eta"]
)
regressor.fit(train_set)

In [ ]:
# Predict on test set
predictions = regressor.predict(test_set)
predictions_pd = predictions.to_pandas()

In [ ]:
# Evaluate model
mse = mean_squared_error(predictions_pd[label_column], predictions_pd["predicted_eta"])
r2 = r2_score(predictions_pd[label_column], predictions_pd["predicted_eta"])
print(f"Test MSE: {mse}")
print(f"Test R2: {r2}")

In [ ]:
# Feature importances
xgb_native = regressor.to_xgboost()

if hasattr(xgb_native, "feature_importances_"):
    importances = xgb_native.feature_importances_
    importance_df = pd.DataFrame({
        'feature': feature_columns,
        'importance': importances
    }).sort_values(by='importance', ascending=False)
    print("Feature importances:")
    print(importance_df)

if hasattr(xgb_native, "get_booster"):
    booster = xgb_native.get_booster()

    weights = booster.get_score(importance_type='weight')
    weights_df = pd.DataFrame(list(weights.items()), columns=['feature', 'weight']).sort_values(by='weight', ascending=False)
    print("Booster weights:")
    print(weights_df)

    gains = booster.get_score(importance_type='gain')
    gains_df = pd.DataFrame(list(gains.items()), columns=['feature', 'gain']).sort_values(by='gain', ascending=False)
    print("Booster gains:")
    print(gains_df)

## Real-time Inference Example
Fetch the latest features from the online store and predict duration for a new trip.

In [ ]:
def predict_trip_duration(pu_location_id, do_location_id, pickup_hour, pickup_day_of_week):
    trip = [
        [pu_location_id, do_location_id, pickup_hour, pickup_day_of_week],
    ]

    # Fetch latest features from the online store
    features_df = fs.read_feature_view(
        registered_route_fv,
        keys=trip,
        store_type=feature_view.StoreType.ONLINE
    )
    
    features_pd = features_df.to_pandas()
    if features_pd.empty:
        print("No online features found, skipping prediction")
        return None
    
    print("Online features:")
    print(features_pd)

    return regressor.predict(features_pd)
    

prediction = predict_trip_duration(141, 236, 8, 0)
if prediction is not None:
    print("Prediction Trip Duration (minutes):")
    print(prediction['predicted_eta'])

## New Data Arrives
Simulate a recently ended trip between the same pickup and dropoff location ids that also occurred on a Sunday between 8am and 9am. In this case, the trip takes longer than normal.

In [ ]:
from datetime import datetime
from snowflake.snowpark.functions import max as sf_max, coalesce, lit

# Compute next TRIP_ID
next_trip_id = (
    session.table(TAXI_TABLE_FULL_NAME)
    .select(coalesce(sf_max("TRIP_ID"), lit(0)) + lit(1))
    .first()[0]
)

# Create single-row DataFrame for the new trip
new_trip_df = session.create_dataframe(
    [
        (
            1,                 # VENDORID
            1,                 # PASSENGER_COUNT
            3.8,               # TRIP_DISTANCE
            1,                 # RATECODEID
            "N",               # STORE_AND_FWD_FLAG
            141,               # PULOCATIONID
            236,               # DOLOCATIONID
            1,                 # PAYMENT_TYPE
            14.50,             # FARE_AMOUNT
            3.00,              # EXTRA
            0.50,              # MTA_TAX
            3.65,              # TIP_AMOUNT
            0.00,              # TOLLS_AMOUNT
            0.30,              # IMPROVEMENT_SURCHARGE
            29.95,             # TOTAL_AMOUNT
            4.5,               # CONGESTION_SURCHARGE
            0,                 # AIRPORT_FEE
            datetime(2025, 8, 31, 8, 55, 0),  # TPEP_PICKUP_DATETIME
            datetime(2025, 8, 31, 9, 15, 0),  # TPEP_DROPOFF_DATETIME
            next_trip_id,      # TRIP_ID
        )
    ],
    schema=[
        "VENDORID",
        "PASSENGER_COUNT",
        "TRIP_DISTANCE",
        "RATECODEID",
        "STORE_AND_FWD_FLAG",
        "PULOCATIONID",
        "DOLOCATIONID",
        "PAYMENT_TYPE",
        "FARE_AMOUNT",
        "EXTRA",
        "MTA_TAX",
        "TIP_AMOUNT",
        "TOLLS_AMOUNT",
        "IMPROVEMENT_SURCHARGE",
        "TOTAL_AMOUNT",
        "CONGESTION_SURCHARGE",
        "AIRPORT_FEE",
        "TPEP_PICKUP_DATETIME",
        "TPEP_DROPOFF_DATETIME",
        "TRIP_ID",
    ],
)

# Append to the existing table
new_trip_df.write.mode("append").save_as_table(TAXI_TABLE_FULL_NAME)
print("Inserted 1 new trip row into", TAXI_TABLE_FULL_NAME)

## Prediction Reflects Latest Data
After a maximum of 75 seconds (60 seconds of data lag for the Dynamic Table and 15 for the Online Feature Table), the latest data has been transformed into its features and is being used for inference. The predicted ETA has increased since the most recently completed trip took longer than normal.

In [ ]:
prediction = predict_trip_duration(141, 236, 8, 0)
if prediction is not None:
    print("Prediction Trip Duration (minutes):")
    print(prediction['predicted_eta'])